# Segmentation Notebook
---
Notebook to experiment / compare various segmentation algorithms.

In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from data.load_data import load_dataset
from source.util import COLOR_TO_HEX, plot_tensors_with_colors

In [5]:
train_dataset = load_dataset(mode='training')
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [6]:
for i, data in enumerate(train_dataloader):
    train = data['train']
    test = data['test']
    
    print(f"Train Samples:\n{'='*100}")
    for t in train:
        plot_tensors_with_colors([t['input'].squeeze(0), t['output'].squeeze(0)])
    
    print(f"Test Samples:\n{'='*100}")
    for t in test:
        plot_tensors_with_colors([t['input'].squeeze(0), t['output'].squeeze(0)])
    break

Train Samples:


NameError: name 'plt' is not defined